In [1]:
import json
import logging
import re
from collections import Counter
import numpy as np
import pandas as pd # Used for easy frequency tables
from transformers import AutoTokenizer
# --- Configuration ---
JSON_FILE_PATH = "saudi_laws_scraped.json"

# Load once and reuse (don’t load inside the function for performance)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-14B")

# Configure logging (kept simple)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

# --- Helper Functions ---

def load_data(file_path):
    """Loads the main JSON data file."""
    logging.info(f"Loading data from {file_path}...")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        logging.info("Data loaded successfully.")
        return data
    except Exception as e:
        logging.critical(f"ERROR loading data: {e}")
        return None


def count_tokens(text):
    """Count tokens using a Hugging Face model tokenizer."""
    if not text:
        return 0
    # The tokenizer.encode method returns a list of token IDs. Its length is the token count.
    return len(tokenizer.encode(text, add_special_tokens=False))


def print_stats(name, data_list):
    """Helper function to print common statistics, including quantiles, for a list of numbers."""
    if not data_list:
        logging.warning(f"No data found for '{name}' statistics.")
        print(f"\n--- Statistics for {name} ---")
        print("  No data available.")
        return

    arr = np.array(data_list)
    print(f"\n--- Statistics for {name} ---")
    
    # Central Tendency and Range
    print(f"  Mean:   {np.mean(arr):,.2f}")
    print(f"  Median: {np.median(arr):,.0f}")
    print(f"  Min:    {np.min(arr):,.0f}")
    print(f"  Max:    {np.max(arr):,.0f}")
    
    # Quantiles
    # The quantiles are calculated using the 'np.quantile' function.
    q25, q75, q90 = np.quantile(arr, [0.25, 0.75, 0.90]) # Q1, Q3, and 90th percentile
    
    print("  --- Quantiles ---")
    print(f"  25th (Q1): {q25:,.0f}")
    print(f"  75th (Q3): {q75:,.0f}")
    print(f"  90th:      {q90:,.0f}")

    # Optional: Interquartile Range (IQR) for dispersion
    iqr = q75 - q25
    print(f"  IQR (Q3-Q1): {iqr:,.0f}")

def run_eda(data):
    """Main function to run the Exploratory Data Analysis with reduced scope, now including Tokens Per Part."""
    
    # --- 1. Initialization ---
    total_laws = 0
    total_parts = 0
    total_articles = 0
    
    # Add subcategory tracking
    subcategory_articles = Counter()
    subcategory_laws = Counter()

    # For Distributions
    parts_per_law_counts = []
    articles_per_law_counts = []
    tokens_per_law_counts = []
    tokens_per_article_counts = []
    articles_per_part_counts = [] 
    tokens_per_part_counts = [] # NEW: Tracking tokens per part
    brief_token_lengths = []
    

    # For Tracking Max/Min
    max_parts_law = {'title': '', 'count': 0}
    max_articles_law = {'title': '', 'count': 0}
    max_tokens_law = {'title': '', 'length': 0}
    max_articles_part = {'law': '', 'part': '', 'count': 0}
    max_tokens_part = {'law': '', 'part': '', 'length': 0} # NEW: Max tokens per part
    longest_article = {'law': '', 'part': '', 'title': '', 'length': 0}
    
    # For Counters
    article_status_counter = Counter()

    # --- 2. Main Processing Loop ---
    logging.info("Starting EDA processing loop...")
    
    for main_cat_name, main_cat_data in data.items():
        for sub_cat_name, sub_cat_data in main_cat_data.items():
            total_laws += len(sub_cat_data)
            subcategory_laws[sub_cat_name] = len(sub_cat_data)
            
            # Initialize article counter for this subcategory
            current_subcat_articles = 0
            
            for law_title, law_data in sub_cat_data.items():
                
                parts = law_data.get('parts', {})
                brief = law_data.get('brief', '')

                # --- Law-Level Counts ---
                num_parts = len(parts)
                parts_per_law_counts.append(num_parts)
                if num_parts > max_parts_law['count']:
                    max_parts_law = {'title': law_title, 'count': num_parts}
                
                brief_token_lengths.append(count_tokens(brief))

                current_law_article_count = 0
                current_law_token_count = 0
                
                for part_title, articles_list in parts.items():
                    total_parts += 1
                    current_subcat_articles += len(articles_list)
                    num_articles_in_part = len(articles_list)
                    articles_per_part_counts.append(num_articles_in_part) 
                    current_law_article_count += num_articles_in_part
                    current_part_token_count = 0 # Initialize part token count
                    
                    if num_articles_in_part > max_articles_part['count']:
                        max_articles_part = {
                            'law': law_title, 
                            'part': part_title, 
                            'count': num_articles_in_part
                        }

                    for article in articles_list:
                        total_articles += 1
                        
                        # --- Article-Level Analysis (Tokens) ---
                        article_text = article.get('Article_Text', '')
                        article_token_len = count_tokens(article_text)
                        
                        tokens_per_article_counts.append(article_token_len)
                        current_law_token_count += article_token_len
                        current_part_token_count += article_token_len # Accumulate for part

                        if article_token_len > longest_article['length']:
                            longest_article = {
                                'law': law_title,
                                'part': part_title,
                                'title': article.get('Article_Title', 'N/A'),
                                'length': article_token_len
                            }
                        
                        article_status = article.get('status', 'Unknown')
                        article_status_counter.update([article_status])
                        
                    # --- Finish Part-Level (Token Counts) ---
                    tokens_per_part_counts.append(current_part_token_count) # Store Part Token Count

                    if current_part_token_count > max_tokens_part['length']:
                        max_tokens_part = {
                            'law': law_title, 
                            'part': part_title, 
                            'length': current_part_token_count
                        }

                
                # --- Finish Law-Level (Article & Token Counts) ---
                articles_per_law_counts.append(current_law_article_count)
                tokens_per_law_counts.append(current_law_token_count) 

                if current_law_article_count > max_articles_law['count']:
                    max_articles_law = {'title': law_title, 'count': current_law_article_count}
                
                if current_law_token_count > max_tokens_law['length']:
                    max_tokens_law = {'title': law_title, 'length': current_law_token_count}

            # Store total articles for this subcategory
            subcategory_articles[sub_cat_name] = current_subcat_articles

    logging.info("...EDA processing loop finished.")
    logging.info("Generating report...")

    # --- 3. Print Report ---
    
    print("=============================================")
    print(" 📊 Concise Exploratory Data Analysis (EDA) Report")
    print("=============================================")

    # --- Section 1: Overall Dataset Statistics ⚖️ ---
    print("\n## 1. Overall Dataset Statistics ⚖️")
    print("---------------------------------------------")
    print(f"  Total Number of Laws:  {total_laws:,}")
    print(f"  Total Number of Parts: {total_parts:,}")
    print(f"  Total Number of Articles: {total_articles:,}")

    # Add new section after Overall Dataset Statistics
    print("\n## 1.1 Subcategory Statistics 📊")
    print("---------------------------------------------")
    
    # Create DataFrame for subcategory statistics
    subcat_stats = pd.DataFrame({
        'Laws': subcategory_laws,
        'Articles': subcategory_articles
    }).sort_values('Articles', ascending=False)
    
    subcat_stats['Articles/Law'] = (subcat_stats['Articles'] / subcat_stats['Laws']).round(1)
    subcat_stats['% of Total Articles'] = (subcat_stats['Articles'] / total_articles * 100).round(1)
    
    print("\nSubcategory Distribution:")
    print(subcat_stats)

    # ---------------------------------------------

    ## 2. Structural Analysis (Law & Part Level) 📑
    
    print("\n## 2. Structural Analysis (Law & Part Level) 📑")
    print("---------------------------------------------")

    # Articles per Part Statistics
    print_stats("Articles per Part", articles_per_part_counts)
    print(f"  Part with the Most Articles: Law: \"{max_articles_part['law']}\", Part: \"{max_articles_part['part']}\" ({max_articles_part['count']} articles)")
    
    # Law-Level Article Stats
    print_stats("Articles per Law", articles_per_law_counts)
    print(f"  Law with Most Articles: \"{max_articles_law['title']}\" ({max_articles_law['count']:,} articles)")
    
    # Parts per Law Stats
    print_stats("Parts per Law", parts_per_law_counts)
    print(f"  Law with Most Parts: \"{max_parts_law['title']}\" ({max_parts_law['count']} parts)")

    # ---------------------------------------------

    ## 3. Token/Length Analysis ✍️
    
    print("\n## 3. Token/Length Analysis ✍️")
    print("---------------------------------------------")
    

    # Tokens per Article Statistics
    print_stats("Tokens per Article", tokens_per_article_counts)
    print("  Longest Single Article (Tokens):")
    print(f"    Law:     \"{longest_article['law']}\"")
    print(f"    Article: \"{longest_article['title']}\"")
    print(f"    Length:  {longest_article['length']:,} tokens")

    # Tokens per Part Statistics (ADDED)
    print_stats("Tokens per Part", tokens_per_part_counts)
    print("  Longest Part (by tokens):")
    print(f"    Law:     \"{max_tokens_part['law']}\"")
    print(f"    Part:    \"{max_tokens_part['part']}\"")
    print(f"    Length:  {max_tokens_part['length']:,} tokens")
    

    # Tokens per Law Stats
    print_stats("Tokens per Law", tokens_per_law_counts)
    print(f"  Longest Law (by tokens): \"{max_tokens_law['title']}\" ({max_tokens_law['length']:,} tokens)")
    
    # Brief Token Stats
    print_stats("Law Brief Token Length", brief_token_lengths)

    # ---------------------------------------------

    ## 4. Article Status ℹ️
    
    print("\n## 4. Article Status ℹ️")
    print("---------------------------------------------")
    
    print("\n  Article Status Distribution:")
    # Using Pandas for a clean percentage table
    article_status_df = pd.DataFrame(
        article_status_counter.items(), 
        columns=['Status', 'Count']
    ).set_index('Status')
    article_status_df['Percentage'] = (article_status_df['Count'] / total_articles * 100).round(2)
    print(article_status_df)
            
    print("\n=============================================")
    print(" ✅ Concise EDA Report Complete")
    print("=============================================")


if __name__ == "__main__":
    # 1. Load the data
    scraped_data = load_data(JSON_FILE_PATH)
    
    # 2. Run the EDA
    if scraped_data:
        run_eda(scraped_data)
    else:
        logging.error("Could not run EDA because data failed to load.")

/home/naif/miniconda3/envs/legal-assistant/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-24 17:49:35,061 - INFO - Loading data from saudi_laws_scraped.json...
2025-10-24 17:49:35,126 - INFO - Data loaded successfully.
2025-10-24 17:49:35,126 - INFO - Starting EDA processing loop...
2025-10-24 17:49:38,035 - INFO - ...EDA processing loop finished.
2025-10-24 17:49:38,035 - INFO - Generating report...


 📊 Concise Exploratory Data Analysis (EDA) Report

## 1. Overall Dataset Statistics ⚖️
---------------------------------------------
  Total Number of Laws:  517
  Total Number of Parts: 2,039
  Total Number of Articles: 16,371

## 1.1 Subcategory Statistics 📊
---------------------------------------------

Subcategory Distribution:
                                                  Laws  Articles  \
التجارة والاقتصاد والاستثمار                        34       506   
الخدمة العسكرية                                      6       253   
الخدمات البلدية والتخطيط والتطوير الحضري             6       214   
الأنظمة الأساسية                                     5       212   
المواصلات والاتصالات                                12       194   
الإعلام والثقافة والنشر                             12       183   
الزراعة والمياه والثروات الحية                      12       175   
الطاقة والصناعة والتعدين                            11       167   
الأمن الداخلي والأحوال المدنية والأنظمة الجنائية     9

In [2]:
def build_search_index(file_path="saudi_laws_scraped.json"):
    """
    Loads the hierarchical JSON and flattens it into a search index 
    where keys are Law Titles.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}

    search_index = {}
    
    # Traverse the 3-level hierarchy
    for main_cat_name, sub_categories in data.items():
        for sub_cat_name, laws in sub_categories.items():
            for law_title, law_data in laws.items():
                search_index[law_title] = law_data
                
    return search_index

LAW_SEARCH_INDEX = build_search_index()


In [3]:
with open("saudi_laws_scraped.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

    search_index = {}
    
    # Traverse the 3-level hierarchy
    for main_cat_name, sub_categories in data.items():
        for sub_cat_name, laws in sub_categories.items():
            search_index[sub_cat_name] = laws

                

In [4]:
[len(search_index[k]) for k in search_index.keys()]

[5, 12, 9, 34, 2, 9, 3, 6, 6, 2, 12, 3, 7, 8, 11, 11, 5, 3, 12, 5]

In [11]:
[ k for k in search_index.keys() if len(search_index[k])<7]

['الأنظمة الأساسية',
 'التشريفات والمراسم والسلك الدبلوماسي',
 'الحج والشؤون الإسلامية',
 'الخدمات البلدية والتخطيط والتطوير الحضري',
 'الخدمة العسكرية',
 'الخدمة المدنية',
 'السلطة القضائية وحقوق الإنسان',
 'العمل والرعاية الاجتماعية',
 'المال والرقابة',
 'التنظيمات الإدارية']

In [15]:
[ {k:list(search_index[k].keys())} for k in search_index.keys() if len(search_index[k])<7]

[{'الأنظمة الأساسية': ['النظام الأساسي للحكم',
   'نظام مجلس الشورى',
   'نظام مجلس الوزراء',
   'نظام المناطق',
   'نظام هيئة البيعة']},
 {'التشريفات والمراسم والسلك الدبلوماسي': ['اللائحة التنفيذية لنظام الأوسمة السعودية',
   'تنظيم مشاركة الوفود الرسمية في الاجتماعات']},
 {'الحج والشؤون الإسلامية': ['تنظيم الرئاسة العامة لهيئة الأمر بالمعروف والنهي عن المنكر',
   'تنظيم مجمع الملك عبدالعزيز للمكتبات الوقفية',
   'تنظيم مجمع خادم الحرمين الشريفين الملك سلمان بن عبدالعزيز ال سعود للحديث النبوي الشريف']},
 {'الخدمات البلدية والتخطيط والتطوير الحضري': ['تنظيم الهيئة العامة للإسكان',
   'تنظيم الهيئة العامة للعقار',
   'تنظيم مؤسسة حديقة الملك سلمان',
   'تنظيم الهيئة العامة لعقارات الدولة',
   'تنظيم هيئات تطوير المناطق والمدن واللائحة المالية لهيئات تطوير المناطق والمدن واللائحة الإدارية لهيئات تطوير المناطق والمدن',
   'الترتيبات التنظيمية للمركز السعودي لكود البناء']},
 {'الخدمة العسكرية': ['نظام التقاعد العسكري',
   'نظام الكليات العسكرية',
   'نظام مجلس الخدمة العسكرية',
   'نظام ا

In [5]:
search_index.keys()

dict_keys(['الأنظمة الأساسية', 'الإعلام والثقافة والنشر', 'الأمن الداخلي والأحوال المدنية والأنظمة الجنائية', 'التجارة والاقتصاد والاستثمار', 'التشريفات والمراسم والسلك الدبلوماسي', 'التعليم والعلوم', 'الحج والشؤون الإسلامية', 'الخدمات البلدية والتخطيط والتطوير الحضري', 'الخدمة العسكرية', 'الخدمة المدنية', 'الزراعة والمياه والثروات الحية', 'السلطة القضائية وحقوق الإنسان', 'السياحة والآثار', 'الشباب والرياضة', 'الصحة', 'الطاقة والصناعة والتعدين', 'العمل والرعاية الاجتماعية', 'المال والرقابة', 'المواصلات والاتصالات', 'التنظيمات الإدارية'])

In [10]:
tokens = tokenizer.encode(f'[{','.join(search_index.keys())}]', add_special_tokens=False)
len(tokens)

159

In [6]:
search_index['الأنظمة الأساسية'].keys() # الأنظمة الأساسية والرئاسية وتنيظم هياكل الدولة

dict_keys(['النظام الأساسي للحكم', 'نظام مجلس الشورى', 'نظام مجلس الوزراء', 'نظام المناطق', 'نظام هيئة البيعة'])

In [7]:
search_index['التنظيمات الإدارية'].keys()

dict_keys(['تنظيم الهيئة الملكية لمحافظة العلا', 'تنظيم هيئة تطوير بوابة الدرعية', 'الترتيبات التنظيمية للهيئة الملكية لمدينة مكة المكرمة والمشاعر المقدسة', 'الترتيبات التنظيمية لجائزة الملك عبد العزيز للجودة', 'الترتيبات التنظيمية لمركز الفعاليات (المركز الوطني للفعاليات)'])

In [8]:
len(LAW_SEARCH_INDEX.keys())

517

In [9]:
tokens = tokenizer.encode(f'[{','.join(LAW_SEARCH_INDEX.keys())}]', add_special_tokens=False)
len(tokens)

6598

In [17]:
LAW_SEARCH_INDEX["نظام البيئة"]['parts']['الفصل الأول ( أحكام عامة )']

[{'id': 6635,
  'Article_Title': 'المادة الأولى',
  'status': 'Modified',
  'Article_Text': 'يقصد بالألفاظ والعبارات الآتية -أينما وردت في هذا النظام- المعاني المبينة أمام كل منها، ما لم يقتض السياق غير ذلك:\nالنظام: نظام البيئة.\nاللوائح: اللوائح التنفيذية للنظام.\nالوزارة: وزارة البيئة والمياه والزراعة.\nالوزير: وزير البيئة والمياه والزراعة.\nالجهة المختصة: الوزارة، أو أي من المراكز الوطنية لقطاع البيئة، كل بحسب اختصاصه، ووفقاً لما تحدده اللوائح.\nالجهة المشرفة: أي جهة حكومية مُخوَّلة نظاماً بالإشراف على قطاع تنموي، وتختص بإصدار تراخيص لممارسة الأنشطة التي تخضع لإشرافها.\nالشخص: أي شخص ذي صفة طبيعية أو اعتبارية عامة أو خاصة.\nالتصريح: وثيقة تمنحها الجهة المختصة للشخص قبل البدء بممارسته أي نشاط له أثر بيئي.\nالترخيص: وثيقة تمنحها الجهة المختصة للشخص للإذن له بممارسة نشاط بيئي.\nالمصرح له: شخص حاصل على التصريح.\nالمرخص له: شخص حاصل على الترخيص.\nنشاط بيئي: أي نشاط تشغيلي أو فني له علاقة بقطاع البيئة.\nالأثر البيئي: كل تغيير سلبي أو إيجابي يؤثر في البيئة نتيجة ممارسة أي نشاط.\nالبيئة / 

In [23]:
len([1 for k in LAW_SEARCH_INDEX if len(LAW_SEARCH_INDEX[k]['parts'].keys())>1])

116